# Baseline: Greedy Placement + Fix Direction + Local Search

This baseline implements:
1. Greedy placement algorithm (from getting-started kernel)
2. Fix direction rotation optimization
3. Simple local search with translation/rotation moves

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# Tree geometry constants
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def generate_weighted_angle():
    """Generates a random angle weighted by abs(sin(2*angle)) for better corner packing."""
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def get_side_length(trees):
    """Get the side length of the bounding square for a list of trees."""
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return max(width, height)

def has_collision(tree, placed_trees, tree_index=None):
    """Check if a tree collides with any placed trees."""
    if not placed_trees:
        return False
    if tree_index is None:
        placed_polygons = [p.polygon for p in placed_trees]
        tree_index = STRtree(placed_polygons)
    else:
        placed_polygons = [p.polygon for p in placed_trees]
    
    possible_indices = tree_index.query(tree.polygon)
    for i in possible_indices:
        if tree.polygon.intersects(placed_polygons[i]) and not tree.polygon.touches(placed_polygons[i]):
            return True
    return False

print("Helper functions defined")

Helper functions defined


In [4]:
def initialize_trees(num_trees, existing_trees=None):
    """Greedy placement algorithm - builds incrementally from previous solution."""
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = [t.clone() for t in existing_trees]

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        unplaced_trees = [
            ChristmasTree(angle=random.uniform(0, 360)) for _ in range(num_to_add)]
        if not placed_trees:
            placed_trees.append(unplaced_trees.pop(0))

        for tree_to_place in unplaced_trees:
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            min_radius = Decimal('Infinity')

            for _ in range(10):  # Try 10 random starting attempts
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    possible_indices = tree_index.query(candidate_poly)
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in

                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py

            tree_to_place.center_x = best_px
            tree_to_place.center_y = best_py
            tree_to_place.polygon = affinity.translate(
                tree_to_place.polygon,
                xoff=float(tree_to_place.center_x * scale_factor),
                yoff=float(tree_to_place.center_y * scale_factor),
            )
            placed_trees.append(tree_to_place)

    side_length = get_side_length(placed_trees)
    return placed_trees, side_length

print("Greedy placement function defined")

Greedy placement function defined


In [5]:
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find the optimal rotation angle to minimize bounding box."""
    if not trees:
        return Decimal('0'), 0.0
    
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)

    hull_points = points_np[ConvexHull(points_np).vertices]

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    improvement = initial_side - found_side
    EPSILON = 1e-8

    if improvement > EPSILON:
        best_angle_deg = found_angle_deg
        best_side = Decimal(str(found_side))
    else:
        best_angle_deg = 0.0
        best_side = Decimal(str(initial_side))

    return best_side, best_angle_deg

def apply_rotation(trees, angle_deg):
    """Apply a rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center / float(scale_factor)
    rotated = shifted.dot(rot_matrix.T) + rotation_center / float(scale_factor)

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation optimization functions defined")

Rotation optimization functions defined


In [6]:
def local_search(trees, max_iterations=100):
    """Simple local search with translation and rotation moves."""
    if len(trees) <= 1:
        return trees, get_side_length(trees)
    
    best_trees = [t.clone() for t in trees]
    best_side = get_side_length(best_trees)
    
    step_sizes = [0.01, 0.004, 0.0015]
    rotation_deltas = [5, 2, 0.8]
    
    improved = True
    iteration = 0
    
    while improved and iteration < max_iterations:
        improved = False
        iteration += 1
        
        for tree_idx in range(len(best_trees)):
            original_tree = best_trees[tree_idx].clone()
            
            # Try translation moves
            for step in step_sizes:
                for dx, dy in [(step, 0), (-step, 0), (0, step), (0, -step),
                               (step, step), (step, -step), (-step, step), (-step, -step)]:
                    test_tree = ChristmasTree(
                        str(float(original_tree.center_x) + dx),
                        str(float(original_tree.center_y) + dy),
                        str(original_tree.angle)
                    )
                    
                    # Check collision with other trees
                    other_trees = best_trees[:tree_idx] + best_trees[tree_idx+1:]
                    if not has_collision(test_tree, other_trees):
                        test_trees = best_trees[:tree_idx] + [test_tree] + best_trees[tree_idx+1:]
                        test_side = get_side_length(test_trees)
                        
                        if test_side < best_side:
                            best_trees = test_trees
                            best_side = test_side
                            improved = True
                            break
                if improved:
                    break
            
            if improved:
                break
            
            # Try rotation moves
            for delta in rotation_deltas:
                for d_angle in [delta, -delta]:
                    test_tree = ChristmasTree(
                        str(original_tree.center_x),
                        str(original_tree.center_y),
                        str(float(original_tree.angle) + d_angle)
                    )
                    
                    other_trees = best_trees[:tree_idx] + best_trees[tree_idx+1:]
                    if not has_collision(test_tree, other_trees):
                        test_trees = best_trees[:tree_idx] + [test_tree] + best_trees[tree_idx+1:]
                        test_side = get_side_length(test_trees)
                        
                        if test_side < best_side:
                            best_trees = test_trees
                            best_side = test_side
                            improved = True
                            break
                if improved:
                    break
            
            if improved:
                break
    
    return best_trees, best_side

print("Local search function defined")

Local search function defined


In [7]:
# Build all configurations from 1 to 200 trees
random.seed(42)  # For reproducibility

all_configs = {}  # n -> list of trees
all_side_lengths = {}  # n -> side length

start_time = time.time()
current_trees = None

for n in range(1, 201):
    # Greedy placement
    trees, side = initialize_trees(n, current_trees)
    
    # Apply fix_direction rotation optimization
    optimized_side, best_angle = optimize_rotation(trees)
    if optimized_side < side:
        trees = apply_rotation(trees, best_angle)
        side = optimized_side
    
    # Simple local search (limited iterations for speed)
    if n <= 50:  # Only do local search for smaller configs to save time
        trees, side = local_search(trees, max_iterations=20)
    
    all_configs[n] = trees
    all_side_lengths[n] = side
    current_trees = trees
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f"n={n}: side={float(side):.6f}, elapsed={elapsed:.1f}s")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.1f}s")

n=20: side=3.752699, elapsed=16.6s


n=40: side=5.592390, elapsed=105.4s


n=60: side=6.815464, elapsed=217.3s


n=80: side=7.592674, elapsed=217.8s


n=100: side=8.991580, elapsed=218.4s


n=120: side=9.624150, elapsed=219.1s


n=140: side=10.695368, elapsed=219.9s


n=160: side=11.051408, elapsed=220.8s


n=180: side=12.173664, elapsed=221.8s


n=200: side=12.515599, elapsed=222.8s

Total time: 222.8s


In [8]:
# Calculate total score
def calculate_score(side_lengths):
    """Calculate total score: sum of s^2/n for all n."""
    return sum(float(s)**2 / n for n, s in side_lengths.items())

total_score = calculate_score(all_side_lengths)
print(f"Total Score: {total_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {total_score - 68.922808:.6f}")

Total Score: 155.669833
Target: 68.922808
Gap: 86.747025


In [9]:
# Validate all configurations for overlaps
def validate_all_configs(configs):
    """Check all configurations for overlaps."""
    invalid = []
    for n, trees in configs.items():
        if len(trees) <= 1:
            continue
        polygons = [t.polygon for t in trees]
        tree_index = STRtree(polygons)
        
        has_overlap = False
        for i, poly in enumerate(polygons):
            indices = tree_index.query(poly)
            for idx in indices:
                if idx == i:
                    continue
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    has_overlap = True
                    break
            if has_overlap:
                break
        
        if has_overlap:
            invalid.append(n)
    
    return invalid

invalid_configs = validate_all_configs(all_configs)
if invalid_configs:
    print(f"WARNING: {len(invalid_configs)} configurations have overlaps: {invalid_configs[:10]}...")
else:
    print("All configurations are valid (no overlaps)")

All configurations are valid (no overlaps)


In [10]:
# Create submission file
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

tree_data = []
for n in range(1, 201):
    for tree in all_configs[n]:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

# Round to 6 decimal places
for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

# Prepend 's' to all values
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(submission.head(20))
print(f"\nTotal rows: {len(submission)}")

                x           y          deg
id                                        
001_0   s0.056435  s-0.305079  s314.999998
002_0   s0.307626  s-0.229489  s346.663915
002_1   s0.002093   s0.119736   s40.667789
003_0    s0.69498    s0.22095  s419.618436
003_1   s0.271533   s0.031207  s113.622309
003_2   s-0.15409    s0.32492  s368.520699
004_0   s0.414143   s0.505316  s463.909546
004_1   s0.242038   s0.075304   s157.91342
004_2  s-0.252225   s-0.01617   s412.81181
004_3   s0.651733  s-0.244783   s182.89507
005_0   s0.345949    s0.52787  s472.390353
005_1   s0.239144   s0.077177  s166.394226
005_2  s-0.236224   s-0.08619  s421.292616
005_3   s0.721565  s-0.128988  s191.375876
005_4    s0.42701   s0.861959  s293.629377
006_0  s-0.344946   s0.347911  s550.691281
006_1   s0.074727   s0.151937  s244.695155
006_2   s0.138309  s-0.346682  s499.593544
006_3   s0.374431   s0.582531  s269.676804
006_4  s-0.655658   s0.495031  s371.930306

Total rows: 20100


In [11]:
# Save submission
submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/001_baseline/submission.csv')
print("Submission saved!")
print(f"\nFinal Score: {total_score:.6f}")

Submission saved!

Final Score: 155.669833


In [12]:
# Load and score the sample submission to compare
def load_submission(filepath):
    """Load a submission CSV and return configs and side lengths."""
    df = pd.read_csv(filepath)
    df['x'] = df['x'].str.strip('s').astype(float)
    df['y'] = df['y'].str.strip('s').astype(float)
    df['deg'] = df['deg'].str.strip('s').astype(float)
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    configs = {}
    side_lengths = {}
    
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [ChristmasTree(center_x=str(row['x']), center_y=str(row['y']), angle=str(row['deg']))
                 for _, row in group_data.iterrows()]
        configs[n] = trees
        side_lengths[n] = get_side_length(trees)
    
    return configs, side_lengths

sample_configs, sample_side_lengths = load_submission('/home/data/sample_submission.csv')
sample_score = calculate_score(sample_side_lengths)
print(f"Sample Submission Score: {sample_score:.6f}")
print(f"My Baseline Score: {total_score:.6f}")
print(f"Target: 68.922808")

Sample Submission Score: 173.652299
My Baseline Score: 155.669833
Target: 68.922808
